In [8]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [1]:
import minsearch
import openai
import json

ModuleNotFoundError: No module named 'openai'

In [ ]:
with open("documents.json", "rt") as f:
    docs_raw = json.load(f)

In [ ]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

sample search 

available course - 
-    data-engineering-zoomcamp
-    machine-learning-zoomcamp
-    mlops-zoomcamp

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
def search(q):
    boost_dict = {"question": 3, "text": 0.5}
    results = index.search(
        q,
        boost_dict=boost_dict,
        num_results=5,
        filter_dict={"course": "data-engineering-zoomcamp"},
    )
    return results

In [ ]:
def create_prompt(q, results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.z

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    context = ""

    for doc in results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=q, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

In [12]:
def rag(q):
    search_results = search(q)
    prompt = create_prompt(q, search_results)
    answer = llm(prompt)
    return answer

In [13]:
q = "is it still a good time to start data engineering zoomcamp. the course has already started."
answer = rag(q)
print(answer)

Yes, you can still join the Data Engineering Zoomcamp after the course has started. You are eligible to submit homework even if you didn't register before the start date. Just be mindful of the deadlines for homework and the final projects.
